In [4]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
import torch.optim as optimizers
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from torch.utils.data import DataLoader
from torchvision import datasets
from torch.utils.data import random_split
import torchvision.transforms as transforms

np.random.seed(123)
torch.manual_seed(123)

In [5]:
class EarlyStopping:
    '''
    早期終了 (early stopping)
    '''
    def __init__(self, patience=0, verbose=0):
        self._step = 0
        self._loss = float('inf')
        self.patience = patience
        self.verbose = verbose

    def __call__(self, loss):
        if self._loss < loss:
            self._step += 1
            if self._step > self.patience:
                if self.verbose:
                    print('early stopping')
                return True
        else:
            self._step = 0
            self._loss = loss

        return False


In [6]:
# Adding Problemのデータを生成
# maskを生成する関数
def mask(T=200):
    mask = np.zeros(T)
    indices = np.random.permutation(np.arange(T))[:2]
    mask[indices] = 1
    return mask

# シミュレーションデータを生成する関数
def toy_problem(N=10, T=200):
    signals = np.random.uniform(0.0, 1.0, size=(N, T))
    masks = np.zeros((N, T))
    for i in range(N):
        masks[i] = mask(T)

    data = np.zeros((N, T, 2))
    data[:, :, 0] = signals[:]
    data[:, :, 1] = masks[:]
    target = (signals * masks).sum(axis=1).reshape(N, 1)
    return data, signals, masks, target

# datasetの生成
N = 10000
T = 200
maxlen = T
x, signals, masks, t = toy_problem(N, T)
#x = signals.reshape(N, T, 1)
x_train, x_val, t_train, t_val = train_test_split(x, t, test_size=0.2, shuffle=False)

In [7]:
# モデルを定義
# RNNを定義
class RNN(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.l1 = nn.RNN(2, hidden_dim, nonlinearity="tanh", batch_first=True)
        self.l2 = nn.Linear(hidden_dim, 1)

        nn.init.xavier_normal_(self.l1.weight_ih_l0)
        nn.init.orthogonal_(self.l1.weight_hh_l0)

    def forward(self, x):
        h, _ = self.l1(x)
        y = self.l2(h[:, -1])
        return y
    
# LSTMを定義
class LSTM(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.l1 = nn.LSTM(2, hidden_dim, batch_first=True)
        self.l2 = nn.Linear(hidden_dim, 1)

        nn.init.xavier_normal_(self.l1.weight_ih_l0)
        nn.init.orthogonal_(self.l1.weight_hh_l0)

    def forward(self, x):
        h, _ = self.l1(x)
        y = self.l2(h[:, -1])
        return y

In [8]:
# モデルの学習
# アルゴリズムの定義
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_rnn = RNN(50).to(device)
model_lstm = LSTM(50).to(device)
criterion = nn.MSELoss(reduction="mean")

# optimizerを定義
optimizer_rnn = optimizers.Adam(model_rnn.parameters(), lr=0.001, betas=(0.9, 0.999), amsgrad=True)
optimizer_lstm = optimizers.Adam(model_lstm.parameters(), lr=0.001, betas=(0.9, 0.999), amsgrad=True)

def compute_loss(t, y):
    return criterion(y, t)

def train_step(x, t, model, optimizer):
    x = torch.Tensor(x).to(device)
    t = torch.Tensor(t).to(device)
    model.train()
    preds = model(x)
    loss = compute_loss(t, preds)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return loss, preds

def val_step(x, t, model):
    x = torch.Tensor(x).to(device)
    t = torch.Tensor(t).to(device)
    model.eval()
    preds = model(x)
    loss = criterion(preds, t)
    return loss, preds

# アルゴリズムの設定
epochs = 100
batch_size = 100
n_batches_train = x_train.shape[0] // batch_size 
n_batches_val = x_val.shape[0] // batch_size 
hist_rnn = {"train_loss": [], "val_loss" : []}
hist_lstm = {"train_loss": [], "val_loss": []}
es = EarlyStopping(patience=10, verbose=1)
disp = 10

In [9]:
# LSTMのパラメータを推定
for epoch in range(epochs):
    
    train_loss = 0.0
    val_loss = 0.0
    x_, t_ = shuffle(x_train, t_train)

    for batch in range(n_batches_train):
        start = batch * batch_size
        end = start + batch_size
        loss, _ = train_step(x_[start:end], t_[start:end], model_lstm, optimizer_lstm)
        train_loss += loss.item()

    for batch in range(n_batches_val):
        start = batch * batch_size
        end = start + batch_size
        loss, _ = val_step(x_val[start:end], t_val[start:end], model_lstm)
        val_loss += loss.item()

    train_loss /= n_batches_train
    val_loss /= n_batches_val

    hist_lstm["train_loss"].append(train_loss)
    hist_lstm["val_loss"].append(val_loss)

    if epoch%disp==0:
        print("epoch: {}, loss: {:.3}, val_loss: {:.3}".format(epoch, train_loss, val_loss))

epoch: 0, loss: 0.362, val_loss: 0.179
epoch: 10, loss: 0.169, val_loss: 0.177
epoch: 20, loss: 0.167, val_loss: 0.177
epoch: 30, loss: 0.167, val_loss: 0.178
epoch: 40, loss: 0.167, val_loss: 0.177
epoch: 50, loss: 0.167, val_loss: 0.177
epoch: 60, loss: 0.172, val_loss: 0.176
epoch: 70, loss: 0.164, val_loss: 0.174
epoch: 80, loss: 0.00561, val_loss: 0.00549
epoch: 90, loss: 0.00269, val_loss: 0.00241


In [11]:
# RNNのパラメータを推定
for epoch in range(epochs):
    
    train_loss = 0.0
    val_loss = 0.0
    x_, t_ = shuffle(x_train, t_train)

    for batch in range(n_batches_train):
        start = batch * batch_size
        end = start + batch_size
        loss, _ = train_step(x_[start:end], t_[start:end], model_rnn, optimizer_rnn)
        train_loss += loss.item()

    for batch in range(n_batches_val):
        start = batch * batch_size
        end = start + batch_size
        loss, _ = val_step(x_val[start:end], t_val[start:end], model_rnn)
        val_loss += loss.item()

    train_loss /= n_batches_train
    val_loss /= n_batches_val

    hist_rnn["train_loss"].append(train_loss)
    hist_rnn["val_loss"].append(val_loss)

    if epoch%disp==0:
        print("epoch: {}, loss: {:.3}, val_loss: {:.3}".format(epoch, train_loss, val_loss))

epoch: 0, loss: 0.168, val_loss: 0.178
epoch: 10, loss: 0.168, val_loss: 0.181
epoch: 20, loss: 0.167, val_loss: 0.178
epoch: 30, loss: 0.167, val_loss: 0.178
epoch: 40, loss: 0.165, val_loss: 0.177
epoch: 50, loss: 0.168, val_loss: 0.178
epoch: 60, loss: 0.167, val_loss: 0.178
epoch: 70, loss: 0.167, val_loss: 0.179
epoch: 80, loss: 0.166, val_loss: 0.177
epoch: 90, loss: 0.164, val_loss: 0.178
